# 🧹 Data Cleaning  – Retail Sales

**Objective**: Clean raw sales data to fix common issues and prepare for analysis.

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv("/Users/anuragchaubey/Online-Retail-Dashboard/data/Online_Retail_data.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     525652 non-null  float64
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    520233 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      482300 non-null  object 
dtypes: float64(3), object(5)
memory usage: 33.1+ MB


In [6]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,525652.000000,520233.000000,406829.000000
mean,9.538179,4.640991,15287.690570
std,221.328312,98.636328,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [7]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity        16257
InvoiceDate         0
UnitPrice       21676
CustomerID     135080
Country         59609
dtype: int64

# Handling the missing values

## Description

In [8]:
import re
import numpy as np

# Step 1: Handle missing values first
df['Description'] = df['Description'].fillna('UNKNOWN_DESCRIPTION')

# Step 2: Remove leading/trailing whitespace
df['Description'] = df['Description'].str.strip()

# Step 3: Convert to uppercase
df['Description'] = df['Description'].str.upper()

# Step 4: Remove internal extra spaces – handle only if string
df['Description'] = df['Description'].apply(
    lambda x: re.sub(r'\s+', ' ', x) if isinstance(x, str) else x
)

# ✅ Check result
print(df['Description'].head())
print(df['Description'].isnull().sum())


0     WHITE HANGING HEART T-LIGHT HOLDER
1                    WHITE METAL LANTERN
2         CREAM CUPID HEARTS COAT HANGER
3    KNITTED UNION FLAG HOT WATER BOTTLE
4         RED WOOLLY HOTTIE WHITE HEART.
Name: Description, dtype: object
0


## ✅ Final Step-by-Step Cleaning for `Quantity`

🔧 **Goals**:
* Ensure values are numeric
* Coerce invalid entries to `NaN`
* Remove zero or negative values *(unless handling returns separately)*
* Drop rows with missing `Quantity`

In [9]:
# Step 1: Convert Quantity to Numeric (invalid entries → NaN)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

# Step 2: Drop rows where Quantity is missing (NaN)
df = df.dropna(subset=['Quantity'])

# Step 3: Filter out rows where Quantity is zero or negative
# (Unless you're saving returns separately — which you're not doing here)
df = df[df['Quantity'] > 0]

# Step 4: Optional – Convert to Integer if it's whole numbers
if (df['Quantity'] % 1 == 0).all():  # check if all values are whole
    df['Quantity'] = df['Quantity'].astype(int)

# ✅ Done — check summary
print("Remaining rows:", len(df))
print("Missing values in Quantity:", df['Quantity'].isnull().sum())
print("Any non-positive Quantity left?:", (df['Quantity'] <= 0).sum())


Remaining rows: 515333
Missing values in Quantity: 0
Any non-positive Quantity left?: 0


## ✅ Final Step-by-Step Cleaning for `UnitPrice`

🔧 **Goals**:
* Ensure values are numeric (handle non-numeric/garbage values)
* Remove negative prices (0 might be valid)
* Drop missing values (`NaN`)
* Final data type: `float

In [10]:
# Step 1: Convert UnitPrice to Numeric (invalid entries → NaN)
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')

# Step 2: Drop rows where UnitPrice is missing
df = df.dropna(subset=['UnitPrice'])

# Step 3: Filter out rows where UnitPrice is negative
df = df[df['UnitPrice'] >= 0]

# Step 4: Ensure it's a float (typically by default, but explicitly if needed)
df['UnitPrice'] = df['UnitPrice'].astype(float)

# ✅ Summary Check
print("Remaining rows:", len(df))
print("Missing UnitPrice values:", df['UnitPrice'].isnull().sum())
print("Negative UnitPrice entries:", (df['UnitPrice'] < 0).sum())


Remaining rows: 510013
Missing UnitPrice values: 0
Negative UnitPrice entries: 0


In [11]:
median_price = df.loc[df['UnitPrice'] > 0, 'UnitPrice'].median()

In [12]:
# Replace 0s with the calculated median
df['UnitPrice'] = df['UnitPrice'].replace(0, median_price)

## CustomerID

* Filling with UNKNOWN_CUSTOMER

In [13]:
import numpy as np

df['CustomerID'] = df['CustomerID'].replace(['NaN', 'nan', 'None'], np.nan)


In [14]:
df['CustomerID'] = df['CustomerID'].astype('string').str.strip()


In [15]:
df['CustomerID'] = df['CustomerID'].fillna('UNKNOWN_CUSTOMER')


## Country

* Filling with Mode

In [16]:
df['Country'].value_counts()

Country
United Kingdom          415533
Germany                   7731
France                    7156
EIRE                      6769
Spain                     2119
Netherlands               2009
Belgium                   1726
Switzerland               1674
Portugal                  1296
Australia                 1037
Norway                     946
Italy                      636
Channel Islands            631
Finland                    579
Cyprus                     533
Sweden                     387
Unspecified                375
Austria                    340
Denmark                    316
Japan                      275
Poland                     275
Israel                     253
Hong Kong                  242
Singapore                  197
Iceland                    161
USA                        157
Canada                     129
Greece                     124
Malta                       93
United Arab Emirates        56
European Community          51
RSA                         48


In [17]:
df.isnull().sum()

InvoiceNo          0
StockCode          0
Description        0
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID         0
Country        56021
dtype: int64

In [18]:
# 1. Treat string 'NaN', 'nan', 'None', etc. as null
df['Country'] = df['Country'].replace(['NaN', 'nan', 'None', 'NONE'], np.nan)

# 2. Strip whitespace and convert to uppercase
df['Country'] = df['Country'].astype(str).str.strip().str.upper()
df['Country'] = df['Country'].replace('NAN', np.nan)  # Because str.upper() turned NaN to 'NAN'

# 3. Get top 3 most frequent countries (excluding NaN)
top_countries = df['Country'].value_counts().head(3).index.tolist()

# 4. Number of missing values
num_missing = df['Country'].isna().sum()

# 5. Compute how many to fill for each
fill_counts = {
    top_countries[0]: int(num_missing * 0.6),
    top_countries[1]: int(num_missing * 0.2),
    top_countries[2]: num_missing - int(num_missing * 0.6) - int(num_missing * 0.2)  # Remaining
}

# 6. Randomly assign top 3 countries into NaNs
missing_indices = df[df['Country'].isna()].index.to_list()
np.random.shuffle(missing_indices)  # Shuffle to avoid bias

fill_idx = 0
for country, count in fill_counts.items():
    selected_indices = missing_indices[fill_idx: fill_idx + count]
    df.loc[selected_indices, 'Country'] = country
    fill_idx += count


In [19]:
df['Country'].value_counts()

Country
UNITED KINGDOM          449145
GERMANY                  18935
FRANCE                   18361
EIRE                      6769
SPAIN                     2119
NETHERLANDS               2009
BELGIUM                   1726
SWITZERLAND               1674
PORTUGAL                  1296
AUSTRALIA                 1037
NORWAY                     946
ITALY                      636
CHANNEL ISLANDS            631
FINLAND                    579
CYPRUS                     533
SWEDEN                     387
UNSPECIFIED                375
AUSTRIA                    340
DENMARK                    316
JAPAN                      275
POLAND                     275
ISRAEL                     253
HONG KONG                  242
SINGAPORE                  197
ICELAND                    161
USA                        157
CANADA                     129
GREECE                     124
MALTA                       93
UNITED ARAB EMIRATES        56
EUROPEAN COMMUNITY          51
RSA                         48


In [20]:
df.duplicated().sum()

np.int64(4427)

### Remove Duplicate Rows

In [21]:
# Get duplicate rows count
duplicate_count = df.duplicated().sum()

# Remove duplicates
df = df.drop_duplicates()

print(f"✅ Removed {duplicate_count} duplicate rows.")


✅ Removed 4427 duplicate rows.


 ### 2. Verify and Fix Data Types

In [22]:
# Convert to correct dtypes
df['Quantity'] = df['Quantity'].astype(int)
df['UnitPrice'] = df['UnitPrice'].astype(float)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Convert object identifiers to string
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df['StockCode'] = df['StockCode'].astype(str)
df['Description'] = df['Description'].astype(str)
df['CustomerID'] = df['CustomerID'].astype(str)
df['Country'] = df['Country'].astype(str)

# Final dtype check
print("\n✅ Data types verified:")
print(df.dtypes)



✅ Data types verified:
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object


In [23]:
# You should track before-after counts during each step
print("\n📋 Summary of Changes:")
print(f"Final shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Remaining duplicate rows: {df.duplicated().sum()}")
print(f"Total missing values:\n{df.isna().sum()}")



📋 Summary of Changes:
Final shape: 505586 rows × 8 columns
Remaining duplicate rows: 0
Total missing values:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [24]:
# Check for missing in important columns
critical_cols = ['InvoiceNo', 'StockCode', 'Quantity', 'UnitPrice', 'InvoiceDate', 'CustomerID', 'Country']
missing_critical = df[critical_cols].isna().sum()

print("\n🔍 Final Missing Values in Critical Columns:")
print(missing_critical)



🔍 Final Missing Values in Critical Columns:
InvoiceNo      0
StockCode      0
Quantity       0
UnitPrice      0
InvoiceDate    0
CustomerID     0
Country        0
dtype: int64


# Checking for Invalid Values

In [25]:
print("🔎 Invalid InvoiceNo:")
print(df[df['InvoiceNo'].isna() | (df['InvoiceNo'].str.strip() == '')])


🔎 Invalid InvoiceNo:
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [26]:
print("🔎 Invalid StockCode:")
print(df[df['StockCode'].isna() | (df['StockCode'].str.strip() == '')])


🔎 Invalid StockCode:
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [27]:
print("🔎 Invalid Descriptions:")
print(df[df['Description'].str.lower().isin(['', 'nan', 'none', 'null'])])


🔎 Invalid Descriptions:
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [28]:
print("🔎 Invalid Quantity (Negative or Zero):")
print(df[df['Quantity'] <= 0])


🔎 Invalid Quantity (Negative or Zero):
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [29]:
print("🔎 Invalid UnitPrice (Zero or Negative):")
print(df[df['UnitPrice'] <= 0])


🔎 Invalid UnitPrice (Zero or Negative):
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [30]:
from datetime import datetime

print("🔎 Invalid InvoiceDate (Future dates):")
print(df[df['InvoiceDate'] > datetime.now()])


🔎 Invalid InvoiceDate (Future dates):
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [31]:
print("🔎 Invalid CustomerID:")
invalid_cust = df['CustomerID'].str.lower().isin(['nan', 'none', 'null'])
print(df[df['CustomerID'].isna() | invalid_cust])


🔎 Invalid CustomerID:
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [32]:
print("🔎 Invalid Country:")
invalid_country = df['Country'].str.lower().isin(['', 'unknown', '?', 'nan'])
print(df[df['Country'].isna() | invalid_country])


🔎 Invalid Country:
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


In [33]:
df.to_csv("/Users/anuragchaubey/Online-Retail-Dashboard/data/Online_Retail_Cleaned.csv",index=False)